In [ ]:
%%capture
! pip install "datasets" "pytorch-lightning" "wandb" "torcheval" "torchmetrics" "clifford"

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer, seed_everything

from torchmetrics.classification import BinaryAccuracy
from torchmetrics.classification import BinaryPrecision
from torchmetrics.classification import BinaryRecall
from torchmetrics.classification import BinaryF1Score

from clifford import Cl


# GATr ARCHITECTURE

---
## Claudio's section
---

---
## Jacopo's section
---

### Auxiliar / dummy

In [ ]:
dummy_mv = {'': 1.0, 'e1': 1.01, 'e2': 1.02, 'e3': 1.03, 'e4': 1.04, 'e12': 2.01, 'e13': 2.02, 'e14': 2.03, 'e23': 2.04, 'e24': 2.05, 'e34': 2.06,  'e123': 3.01, 'e124': 3.02, 'e134': 3.03,  'e234': 3.04, 'e1234': 5.0}

In [ ]:
def compose_multivector(sample):
    wss_multivector = 0
    for row in sin_sample['wss'][()]:
        vector = row[0] * blades[''] + row[1] * blades[''] + row[2] * blades['']
        wss_multivector += vector

    # pressure as a pseudoscalar
    pressure_multivector = 0
    for elem in sin_sample['pressure'][()]:
        vector = elem * blades['e1234']
        pressure_multivector += vector

      # pos as a point
    pos_multivector = 0
    for row in sin_sample['pos'][()]:
        vector = row[0] * blades['e123'] + row[1] * blades['e124'] + row[2] * blades['e134']
        pos_multivector += vector

      # face as a plane
    face_multivector = 0
    for row in sin_sample['face'][()]:
        vector = row[0] * blades['e1'] + row[1] * blades['e2'] + row[2] * blades['e3']
        face_multivector += vector

      # inlet as a scalar
    inlet_multivector = 0
    for elem in sin_sample['inlet_idcs'][()]:
        vector = elem * blades['']
        inlet_multivector += vector


    total_multivector = wss_multivector + pressure_multivector + pos_multivector + face_multivector + inlet_multivector
    return total_multivector

### Layers

In [ ]:
class J_norm_linear(pl.LightningModule):
    def __init__(self, normalized_shape = (5,), eps=1e-05):
        super(J_norm_linear, self).__init__()

        self.normalized_shape = normalized_shape # shape of the tesnor
        self.eps = eps

    def forward(self, multivector):
        output = torch.nn.functional.layer_norm( multivector, self.normalized_shape, eps = self.eps )

        return output

my_layer = J_norm_linear()

In [ ]:
class J_equi_linear(pl.LightningModule):
    def __init__(self, mv_dict_type, in_feat=5, out_feat=5, ):
        super(J_equi_linear, self).__init__()

        self.mv_dict_type = mv_dict_type # indicate if the multivector is expressed as 16 basis or they are all summed up
        self.equi_linear_layer = torch.nn.Linear(in_features=in_feat, out_features=out_feat)

    def forward(self, multivector):

        if self.mv_dict_type == True: # first equivariant layer of the architecture
            multivector_basis = list( multivector.keys() )

            # summing up all scalars - vectorl - bivectorl - trivectorl - bias ( its '' basis )
            bias = 0                     # should be: 1.0
            vector_value = 0             # should be: 4.1
            bivector_value = 0           # should be: 12.21
            trivector_value = 0          # should be: 12.10
            pseudoscalar_value = 0       # should be: 5.0

            for basis in multivector_basis:
                if len(basis) == 2:     vector_value += multivector[basis]
                elif len(basis) == 3:   bivector_value += multivector[basis]
                elif len(basis) == 4:   trivector_value += multivector[basis]
                elif len(basis) == 5:   pseudoscalar_value += multivector[basis]
                else:                   bias += multivector[basis]

            # check correctness of position of each entries. Maybe switch bias and scalar value (?)
            input_tensor = torch.tensor( [ bias, vector_value, bivector_value, trivector_value, pseudoscalar_value ] ,
                                            dtype=torch.float32,
                                            requires_grad=True
                                        )

            # check why print 12.2099999 instead of 12.1, can be a problem ?
            print(f"--Analizing input component:\nbias(''): {bias}\nvector(e_i): {vector_value}\nbivector(e_ij): {bivector_value}\ntrivector(e_ijk): {trivector_value}\npseudoscalar(e_ijkl): {pseudoscalar_value}")
            print(f"\ninput_tensor: {input_tensor}\n--input's analisys finished. \n\n")
        else:
            input_tensor = multivector

        output = self.equi_linear_layer( input_tensor )

        return output


my_layer = J_equi_linear(mv_dict_type=True)
out = my_layer(dummy_mv)
print(out)

--Analizing input component:
bias(''): 1.0
vector(e_i): 4.1000000000000005
bivector(e_ij): 12.209999999999999
trivector(e_ijk): 12.099999999999998
pseudoscalar(e_ijkl): 5.0

input_tensor: tensor([ 1.0000,  4.1000, 12.2100, 12.1000,  5.0000], requires_grad=True)
--input's analisys finished. 


tensor([-3.9347,  7.6382, -1.1533,  3.5433, -7.7075], grad_fn=<ViewBackward0>)


In [ ]:
class J_gatr(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # input layer: in = input
        self.in_equi_linear_layer = J_equi_linear(in_feat=5, out_feat=5, mv_dict_type=True )

        # first half block: h1 = first half
        self.h1_norm_linear_layer   = J_norm_linear(normalized_shape=(5,))
        self.h1_equi_linear_layer_1 = J_equi_linear(in_feat=5, out_feat=5, mv_dict_type=False )
        #self.h1_att_layer
        #self.h1_equi_linear_layer_2 = J_equi_linear(in_feat=?, out_feat=?, mv_dict_type=? )

        # second half block: h2 = second half
        #self.h2_norm_linear_layer   = J_norm_linear(normalized_shape=?)
        #self.h2_equi_linear_layer_1 = J_equi_linear(in_feat=?, out_feat=?, mv_dict_type=? )
        #self.h2_geo_bilinear_layer  =
        #self.h2_gated_relu_layer    =
        #self.h2_equi_linear_layer_2 = J_equi_linear(in_feat=?, out_feat=?, mv_dict_type=? )


        # metrics
        self.accuracy_metric  = BinaryAccuracy()
        self.precision_metric = BinaryPrecision()
        self.recall_metric    = BinaryRecall()
        self.f1score_metric   = BinaryF1Score()


    def forward(self, multivector):
        do_print = True

        print(f"input: {multivector.values()}\n\nFirst block")
        # input layer
        in_out_eq  = self.in_equi_linear_layer(multivector);       print(f"in_out_eq: {in_out_eq}")

        # first half block
        h1_out_norm = self.h1_norm_linear_layer(in_out_eq);        print(f"h1_out_norm: {h1_out_norm}")
        h1_out_eq_1 = self.h1_equi_linear_layer_1(h1_out_norm);    print(f"h1_out_eq_1: {h1_out_eq_1}")


        # second half block
        print(f"\nSecond block")


        # output ( random for print)
        output = h1_out_eq_1 + h1_out_norm; print(f"\ngatr_output: {out}")

        return output

    def training_step(self, batch):
        x, y = batch

        # check if the used loss is correct according to the paper, find it on the github
        loss = F.binary_cross_entropy(x, y) # In case we need to adjust dimension
        #loss = F.binary_cross_entropy(self(x).view(-1), y.float())

        #predictions = self.forward(x).long().squeeze()
        #y = torch.tensor(y, dtype=torch.long)

        #accuracy  = accuracy(predictions, y)
        #precision = self.precision_metric(predictions, y)
        #recall    = self.recall_metric(predictions, y)
        #f1_score  = self.f1_metric(predictions, y)

        #wandb.log({"acc": accuracy,"loss": loss,"precision": precision, "recall": recall, "f1-score:":f1_score })

        return loss

    '''
    # look at https://colab.research.google.com/drive/1lUIrtEiQN9hA5RIdKTpUS7w5gI1DLvOl#scrollTo=gigZq4h0yifA
    def validation_step(self, batch, batch_idx):
        return None

    def test_step(self, batch, batch_idx):
        return None
    '''
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

gatr_j = J_gatr()

In [ ]:
out = gatr_j(dummy_mv)
print(f"\noutput: {out}")

input: dict_values([1.0, 1.01, 1.02, 1.03, 1.04, 2.01, 2.02, 2.03, 2.04, 2.05, 2.06, 3.01, 3.02, 3.03, 3.04, 5.0])

First block
--Analizing input component:
bias(''): 1.0
vector(e_i): 4.1000000000000005
bivector(e_ij): 12.209999999999999
trivector(e_ijk): 12.099999999999998
pseudoscalar(e_ijkl): 5.0

input_tensor: tensor([ 1.0000,  4.1000, 12.2100, 12.1000,  5.0000], requires_grad=True)
--input's analisys finished. 


in_out_eq: tensor([1.2447, 5.3071, 2.8304, 1.9144, 3.2210], grad_fn=<ViewBackward0>)
h1_out_norm: tensor([-1.1963,  1.7334, -0.0527, -0.7133,  0.2289],
       grad_fn=<NativeLayerNormBackward0>)
h1_out_eq_1: tensor([ 0.6397,  0.6114, -0.9176, -0.8427, -0.9453], grad_fn=<ViewBackward0>)

Second block

gatr_output: tensor([-3.9347,  7.6382, -1.1533,  3.5433, -7.7075], grad_fn=<ViewBackward0>)

output: tensor([-0.5566,  2.3448, -0.9703, -1.5560, -0.7164], grad_fn=<AddBackward0>)


---
## Lorenzo's section
---